[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-08/exercise-01.ipynb)

# 🧪 Exercise 2 — Build Self-Attention From Scratch

## Goal

Make attention feel mechanical.

In [3]:
import sys
!{sys.executable} -m pip install torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 MB 22.7 MB/s eta 0:00:0000:0100:01
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached markupsafe-3.0.3-cp310-cp310-macosx_11_0_arm64.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [torch]32m7/8 [torch]]x]

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip


In [4]:
import torch
import torch.nn.functional as F

batch = 1
seq = 4
dim = 8

x = torch.randn(batch, seq, dim)

Wq = torch.randn(dim, dim)
Wk = torch.randn(dim, dim)
Wv = torch.randn(dim, dim)

Q = x @ Wq
K = x @ Wk
V = x @ Wv

scores = Q @ K.transpose(-2, -1) / dim**0.5
weights = F.softmax(scores, dim=-1)

output = weights @ V

print("Attention matrix:")
print(weights[0])

Attention matrix:
tensor([[1.0000e+00, 7.6036e-12, 1.7051e-06, 8.4510e-11],
        [5.2877e-03, 9.9469e-01, 3.3425e-06, 1.7669e-05],
        [4.2503e-12, 1.9294e-13, 1.0000e+00, 9.4086e-08],
        [1.7502e-07, 1.0000e+00, 7.7696e-12, 1.8689e-06]])


Each row is a routing decision.

Each token decides how much every other token matters.

## Modify One Token

Change token 0 slightly.

In [5]:
# Modify token 0
x_modified = x.clone()
x_modified[0, 0] += 0.5  # Small change to first token

Q_modified = x_modified @ Wq
K_modified = x_modified @ Wk
V_modified = x_modified @ Wv

scores_modified = Q_modified @ K_modified.transpose(-2, -1) / dim**0.5
weights_modified = F.softmax(scores_modified, dim=-1)

print("Original attention matrix:")
print(weights[0])
print("\nModified attention matrix:")
print(weights_modified[0])
print("\nDifference:")
print((weights - weights_modified)[0])

Original attention matrix:
tensor([[1.0000e+00, 7.6036e-12, 1.7051e-06, 8.4510e-11],
        [5.2877e-03, 9.9469e-01, 3.3425e-06, 1.7669e-05],
        [4.2503e-12, 1.9294e-13, 1.0000e+00, 9.4086e-08],
        [1.7502e-07, 1.0000e+00, 7.7696e-12, 1.8689e-06]])

Modified attention matrix:
tensor([[1.0000e+00, 8.3065e-25, 4.7873e-17, 1.0239e-24],
        [3.5539e-05, 9.9994e-01, 3.3602e-06, 1.7763e-05],
        [1.4553e-13, 1.9294e-13, 1.0000e+00, 9.4086e-08],
        [1.3774e-12, 1.0000e+00, 7.7696e-12, 1.8689e-06]])

Difference:
tensor([[-1.6689e-06,  7.6036e-12,  1.7051e-06,  8.4510e-11],
        [ 5.2521e-03, -5.2521e-03, -1.7649e-08, -9.3296e-08],
        [ 4.1048e-12,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.7501e-07, -1.1921e-07, -8.6736e-19, -2.2737e-13]])


**Observe attention matrix changes everywhere.**

This demonstrates contextualization.